In [1]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
train_orders = pd.read_pickle('./otto_train_orders.pkl')
test_clicks = pd.read_pickle('./otto_test_clicks.pkl')
test_carts = pd.read_pickle('./otto_test_carts.pkl')
test_orders = pd.read_pickle('./otto_test_orders.pkl')
# submis = pd.read_csv('./sample_submission.csv')
test =  pd.concat([test_clicks, test_carts, test_orders], axis=0)

In [2]:
test.to_csv('otto_test.csv', index=False)

In [8]:
print(len(test.session.unique())*3)
print(len(test_clicks.session.unique()))

5015409
1670191


In [11]:
test_clicks.columns

Index(['session', 'aid', 'type', 'rating', 'ts'], dtype='object')

In [2]:
cart_samples = test_clicks.groupby(['session']).apply(lambda d: d['aid'].astype(str).tolist()).to_dict()
order_samples = test_carts.groupby(['session']).apply(lambda d: d['aid'].astype(str).tolist()).to_dict()
# for u in tqdm(test_clicks['session'].unique().tolist()):
#     udf = test_clicks[test_clicks['session'] == u]
#     order_samples[u] = list(set(udf.aid.tolist()))
    
    

In [29]:
order_samples

12899779


In [34]:
from pathlib import Path
import pickle

paths = ['./otto_test_nsample_for_clicks.pkl', './otto_test_nsample_for_carts.pkl']

path = Path('./otto_test_nsample_for_carts.pkl')
with path.open("wb") as f:
    pickle.dump(order_samples, f)

In [3]:
train_orders.aid.value_counts()

231487     4485
166037     3824
1733943    3042
1445562    2998
1022566    2788
           ... 
1423851       1
1379439       1
137699        1
1035856       1
1382071       1
Name: aid, Length: 657940, dtype: int64

In [5]:
temp = train_orders.groupby(['type','aid'])['session'].agg('count').reset_index()
temp

,type,aid,session
0,orders,3,38
1,orders,10,1
2,orders,11,3
3,orders,14,3
4,orders,16,2
...,...,...,...
657935,orders,1855592,3
657936,orders,1855593,1
657937,orders,1855594,82
657938,orders,1855597,1


In [6]:
temp.columns = ['type','aid','count']
order_num_df = temp.loc[(temp['type'] == 'orders'), ]
order_num_df = order_num_df.sort_values(['count'],ascending=False).reset_index()
order_num_df

,index,type,aid,count
0,81987,orders,231487,4485
1,58924,orders,166037,3824
2,614879,orders,1733943,3042
3,512359,orders,1445562,2998
4,362170,orders,1022566,2788
...,...,...,...,...
657935,357825,orders,1010520,1
657936,357824,orders,1010518,1
657937,357822,orders,1010505,1
657938,357819,orders,1010497,1


In [10]:
top20 = train_orders.aid.value_counts()[:20].index.tolist()
top20 = [str(i) for i in top20]

In [11]:


session_type = []
labels = []
types = ["clicks", "carts", "orders"]

for i in tqdm(test.session.unique().tolist()):
    for t in types:
        session_type.append(str(i) + "_" + str(t))
        # labels.append(' 80222 1022566 166037 1629608 1733943 332654 351335 923948 1603001 544144 1083665 832192 29735 231487 563117 247240 673407 125278 800391 527209')
        labels.append(' '.join(top20))

100%|██████████| 1671803/1671803 [00:03<00:00, 478265.21it/s]


In [12]:
df = pd.DataFrame({"session_type": session_type, "labels": labels})
df

,session_type,labels
0,12899779_clicks,231487 166037 1733943 1445562 1022566 801774 1...
1,12899779_carts,231487 166037 1733943 1445562 1022566 801774 1...
2,12899779_orders,231487 166037 1733943 1445562 1022566 801774 1...
3,12899780_clicks,231487 166037 1733943 1445562 1022566 801774 1...
4,12899780_carts,231487 166037 1733943 1445562 1022566 801774 1...
...,...,...
5015404,14559742_carts,231487 166037 1733943 1445562 1022566 801774 1...
5015405,14559742_orders,231487 166037 1733943 1445562 1022566 801774 1...
5015406,14568250_clicks,231487 166037 1733943 1445562 1022566 801774 1...
5015407,14568250_carts,231487 166037 1733943 1445562 1022566 801774 1...


In [38]:
df.to_csv("submission.csv", index=False)